In [ ]:
%matplotlib inline
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate, integrate

In [ ]:
# EXAMPLE FUNCTION
f__0 = 0
f__T = 3 * np.pi
r = 3
rho = 10e-4

def f(x):
    def raw_f(x):
        if 0 <= x < np.pi:
            return np.sin(x)
        if np.pi <= x <= 3 * np.pi:
            return np.sin(x - np.pi)

    if isinstance(x, (list, np.ndarray)):
        return [raw_f(elem) for elem in x]

    return raw_f(x)

In [ ]:
mesh = np.linspace(f__0, f__T, 50, dtype='float64')
plt.scatter(mesh, f(mesh))

In [ ]:
# EXAMPLE FUNCTION
f__0 = 0
f__T = 2 * np.pi + 0.5
r = 3
rho = 10e-4

def f(x):
    def raw_f(x):
        if 0 <= x < np.pi:
            return np.sin(x - np.pi)
        if np.pi <= x <= 2 * np.pi + 0.5:
            return np.sin(x - np.pi - 0.5)

    if isinstance(x, (list, np.ndarray)):
        return [raw_f(elem) for elem in x]

    return raw_f(x)

In [ ]:
mesh = np.linspace(f__0, f__T, 50, dtype='float64')
plt.scatter(mesh, f(mesh))

In [ ]:
def A_test(f, a0, a1, b1, b0, r, rho):
    knots = np.linspace(b1, b0, r+1, endpoint=True)
    values = np.array([f(x) for x in knots])
    w1 = interpolate.interp1d(knots, values, fill_value="extrapolate")
    
    knots = np.linspace(a0, a1, r+1, endpoint=True)
    values = np.array([f(x) for x in knots])
    w2 = interpolate.interp1d(knots, values, fill_value="extrapolate")
    
    z_arr = np.linspace(a1, b1, r+1, endpoint=True) # "endpoint=True" is 100% good here
    values = [(np.abs(w1(z_i) - w2(z_i))) / ((b0 - a0) ** (r + rho)) for z_i in z_arr]
    
    return np.max(values)

In [ ]:
def step1(g, c, d, r, rho):
    max_diam = np.max([c[i+1] - c[i] for i in range(len(c)-1)])
    
    if max_diam <= 4 * d:
        return 0, 0
    else:
        largest_result = 0
        second_largest_result = 0
        lagrest_result_index = 0

        for i in range(len(c)-1):
            if c[i+1] - c[i] > 4 * d:
                test_result = A_test(g, c[i], c[i] + d, c[i+1] - d, c[i+1], r, rho)

                if test_result > largest_result:
                    largest_result = test_result
                    lagrest_result_index = i
                elif largest_result > test_result > second_largest_result:
                    second_largest_result = test_result
        
        if math.isclose(largest_result, second_largest_result, rel_tol=1e-7):
            return 0, 0

    return c[lagrest_result_index], c[lagrest_result_index+1]

In [ ]:
def step2(g, a, b, d, r, rho): # bisection
    if a == 0 and b == 0:
        return []
    a_new, b_new = a, b
    B = [a, b]

    while True:
        if b_new - a_new <= 4 * d:
            return B
        
        v = (a_new + b_new) / 2
        A1 = A_test(g, a_new, a_new +  d, v - d, v, r, rho)
        A2 = A_test(g, v, v +  d, b_new - d, b_new, r, rho)
        B.append(v)

        if math.isclose(A1, A2, rel_tol=1e-14):  # TODO: it should depend on precision
            return B
        elif A1 > A2:
            b_new = v    
        else:
            a_new = v

In [ ]:
def step3(g, c, B, d):
    M = np.concatenate((c, B))
    M = np.sort(M)

    def adaptive_approximate(t):
        # locate knot that is smaller or equal to t (and it is closest to t)
        for i in range(len(M)-1):
            if t < M[i+1]:
                break
                
        if M[i+1] - M[i] <= 4 * d:
            return g(M[i])
        else:
            if M[i] <= t < M[i] + d:
                return g(M[i])
            if M[i] + d <= t < M[i+1] - d:
                left, right = M[i] + d, M[i+1] - d
                knots = np.linspace(left, right, r+1, endpoint=True)
                values = np.array([g(x) for x in knots])
                polynomial = interpolate.interp1d(knots, values)
                return polynomial(t)
            if M[i+1] - d <= t < M[i+1]:
                return g(M[i+1] - d)

        if math.isclose(t, M[-1]):
            return g(M[-1])

        return -1
    return M, adaptive_approximate

In [ ]:
def Alg2014(g, a, b, m, r, rho):
    '''
    g - funkcja, którą aproksymujemy
    r - regularność funkcji
    a, b - brzegi przedziału [a,b]
    m - początkowa gęstość siatki
    '''
    # Step 0 (inicjalizacja stałych itp.)
    h = (b - a) / m
    d = h ** (r + rho)
    c = np.linspace(a, b, m+1, dtype='float64', endpoint=True)

    # kroki algorytmu
    # Krok 1: Lokalizowanie osobliwosci
    left, right = step1(g, c, d, r, rho)

    # Krok 2: Wybieranie dodatkowych punktów metoda bisekcji
    B = step2(g, left, right, d, r, rho)
    
    # Krok 3: konstruowanie finalnej aproksymacji
    M, approx = step3(g, c, B, d)
    
    return M, approx


In [ ]:
m = 200

M, f_approximation = Alg2014(f, f__0, f__T, m, r, rho)

In [ ]:
def f_values(mesh, fun):
    return [fun(knot) + 0.1 for knot in mesh]

In [ ]:
plt.figure(figsize=(16, 8))

plt.subplot(3, 1, 1) # row 1, col 2 index 1
plt.scatter(M, f_values(M, f), color="orange", s=1)
plt.scatter(M, f_values(M, f_approximation), s=1)

plt.subplot(3, 1, 2) # index 2
mesh = np.linspace(f__0, f__T, num=m*2, endpoint=True)
plt.scatter(mesh, f_values(mesh, f), color="orange", s=1)
plt.scatter(mesh, f_values(mesh, f_approximation), s=1)

In [ ]:
def worst_case_error(fun, algorithm, alg_data, num, p=2):
    '''
    calculate worst case error with respect to noise for function fun
    '''
    f__a, f__b, m, r, rho = [value for value in alg_data]
    norms = []
    errors = []

    for i in range(num):
        f_approximation = algorithm(f, f__a, f__b, m, r, rho)[1]
        result, error = integrate.quad(lambda x: abs(fun(x) - f_approximation(x))**p, f__a, f__b)
        norm = result**(1/p)
        norms.append(norm)
        errors.append(error) # ERORRS OF INTEGRATION!!! not errors of approximation

    print("norms: ")
    print(["{:.12f}, ".format(value) for value in norms])
    # print("errors: ")
    # print(["{:.13f}, ".format(value) for value in errors])

    return np.max(norms)

In [ ]:
m = 100

worst_case_error(
    fun=f, 
    algorithm=Alg2014,
    alg_data=(f__0, f__T, m, r, rho),
    num=5)